# Components

Smart contract with sensors connected as light nodes that operate governed by the contracts.

Use of MPC for secure data processing. (sending totals)

A PKI will be used to help ensure authenticity of the data being transmitted by sensors. Private keys will be held locally by each sensor (implemented in smart contract).

Data will be stored on the blockchain, that is maintained by the owner.


**Trash Data Logging**
 
Sensors log amount of trash collected every minute and add it to its days aggregate total.

Will return as much as 21.6 lbs/day, around the range of the dashboard shown in the project description

In [ ]:
import random

def SensorCollection(): 
  aggregate = 0 # Total Trash for Day in lbs
  for i in range(1440): # Simulates Data Collected per Minute
    aggregate += random.uniform(0, 0.015)
  return aggregate 
  
print(SensorCollection())

10.763726548668213


**Energy Data Logging**
 
Sensors log amount of energy used every minute and add it to its days aggregate total. Here, we calculate the energy that goes through the sensor.

The average building uses 22.5 kilowatt-hours per square foot per year (based on link below). Assuming that CCDS is 6000 sq.ft (estimate) and 18 floors, we can calculate a daily energy consumption of:

(6000 * 22.5 * 18 ) / 24 = 6,657.53 kilowatt-hours per day.

Therefore, in our model we can estimate each sensor to be seeing energy consumption of around 1,331.506 kWh. This would mean, ~0.92 kWh per minute. We will estimat the data range to be within 0 and 1.4 kWh per minute.

**Note: View modifications to the code section in the Report**


https://www.buildingsiot.com/blog/building-energy-consumption-breakdown-for-owners-and-management-bd#:~:text=According%20to%20Department%20of%20Energy,kilowatt%2Dhours%20per%20square%20foot.

In [ ]:
def SensorEnergyCollection(): 
  aggregate = 0 # Total Energy for Day in KwH
  for i in range(1440): # Simulates Data Collected per Minute
    aggregate += random.uniform(0, 1.4)
  return aggregate 
  
print(SensorEnergyCollection())

986.5457692900062


**Temperature Data Logging**

Temperature is handled a little differently, a single piece of data posted at the end cannot be a total as the total as a data point would not make sense to a user. Therefore, we calculate the total and divide by the number of data points to come up with the day's average temperature.

In [ ]:
def SensorTemperatureCollection(): 
  aggregate = 0 # Total Tem for Day in KwH
  for i in range(1440): # Simulates Data Collected per Minute
    aggregate += random.uniform(0, 100)
  return aggregate/1440
  
print(SensorTemperatureCollection())

50.52054328958682


**MPC Component**
 
Sensors will share their totals through MPC to come up with a daily aggregate total. (Base Code from HW6) This implementation works for Trash Data. It could be modified to meet the needs of energy and temperature data.

Modified SynchronousNetwork and MPCParty to accomodate for the use of decimals for increased precision (correct upto 3 decimal places)

In [ ]:
# Execute this code block if using Google Colab, or else install this package yourself
!pip install fixedint

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
## Execute, but DO NOT MODIFY this code block ##

from typing import List
import random
from fixedint import UInt8

# this method is used only for creating the debug log
def printList(x: List[UInt8]):
    if(x == None):
        return '[]'
    output = '[ '
    for i in x:
        output += str(i) + " "
    output += ']'
    return output

# The Party class is a base class representing a single participant in a distributed protocol.
# This party can choose what messages to send, and can record the contents of messages received.
class Party:
    def __init__(self, id: int, nb_faulty_parties: int, nb_total_parties: int, input: List[UInt8]):
        # Precondition: we guarantee that the party's id is between 1 and n
        self.id = id
        # In your code, you should assume that the party is honest
        # (we handle faulty parties separately in the SynchronousNetwork class)
        self.input = input
        self.is_faulty = False
        self.f = nb_faulty_parties
        self.n = nb_total_parties

    # send and receive proceeds in rounds
    # our SynchronousNetwork always completes one round before starting the next one
    def send(self, round_number: int, destination_party: int) -> List[UInt8]:
        return

    def receive(self, round_number: int, sender_party: int, val: List[UInt8]) -> None:
        # do an action based on what you receive from the sender party
        # but don't return anything
        return None

    # the output method is only run once at the end of the protocol
    # after all rounds of communication are complete
    def output(self):
        ## return what this party decides to output
        return

class SynchronousNetwork:
    # faulty parties only matter for the optional task
    def __init__(self, PartyType, nb_faulty_parties: int,
                 nb_total_parties: int, inputs: list, debug=False):
        # verify that f < n
        assert(nb_faulty_parties < nb_total_parties)
        assert(len(inputs) == nb_total_parties)
        self.debug = debug

        # creating several parties with the prescribed inputs
        self.parties = [PartyType(i, nb_faulty_parties, nb_total_parties,
                                  inputs[i-1]) for i in range(1, nb_total_parties + 1)]
        self.debug_print("Inputs:")
        for i in range(1, nb_total_parties + 1):
            self.debug_print("Party " + str(i) + " input: " + str(inputs[i-1]))

        # randomly set some of the parties to be faulty, except the last one
        for p in random.sample(self.parties, nb_faulty_parties):
            if(p.id != 4):
                p.is_faulty = True
        self.leader = random.sample(self.parties, 1)[0]

    def run(self, nb_rounds) -> list:
        # execute all nb_rounds rounds of the synchronous protocol, in order
        for i in range(1, nb_rounds + 1):
            self.debug_print("\nStart of round " + str(i) + ":")

            for p1 in self.parties:
                for p2 in self.parties:
                    # each party can send one message to all parties (including itself!)
                    val = SynchronousNetwork.send_with_errors(i, p1, p2.id)
                    self.debug_print("party with id " + str(p1.id) + " sending to party with id " 
                                         + str(p2.id) + " a message with content: " + str(val))

                    # recipient receives the message instantaneously
                    p2.receive(i, p1.id, val)

        # after all rounds are finished, retrieve each party's output
        res = [p.output() for p in self.parties]
        return res

    def send_with_errors(round_nb, sender, recv) -> List[UInt8]:
        message = sender.send(round_nb, recv)

        # for honest parties or most of the time for faulty parties,
        # call the sender party and perform the action it wants
        r = random.random()
        if (not sender.is_faulty or r < 0.95):
            return message
        # for faulty parties, occasionally send random numbers
        # this case only matters for the optional task
        else:
            for i in range(len(message)):
                message[i] = UInt8(random.randrange(256))
            return message

    # pretty-printer that you can use to view the network communication
    def debug_print(self, obj):
        if self.debug:
            print(obj)
        else:
            pass

In [ ]:
def splitnum(x):
  while True:
    int1 = random.uniform(0, 255)
    int2 = random.uniform(0, 255)
    int3 = random.uniform(0, 255)
    int4 = random.uniform(0, 255)
    int5 = (x - int1 - int2 - int3 - int4) % 256
    if (int1 and int2 and int3 and int4 and int5) != 0:
      return [int1, int2, int3, int4, int5]

class MPCParty(Party):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.shares = [0] * 5
        self.othershares = [0] * 5

    def send(self, round_number: int, destination_party: int):
        if round_number == 1:
          if self.shares[4] == 0:
            self.shares = splitnum(self.input[0])
          return self.shares[destination_party-1]

    def receive(self, round_number: int, sender_party: int, val: List[UInt8]):
        if round_number == 1:
          self.othershares[self.id-1] = self.shares[self.id-1]
          self.othershares[sender_party-1] = val

    def output(self):
        return sum(self.othershares) % 256

In [ ]:
mpc_inputs = [[SensorCollection()],[SensorCollection()],[SensorCollection()], [SensorCollection()], [SensorCollection()]]
mpcNetwork = SynchronousNetwork(MPCParty, 0, 5, mpc_inputs, debug=True)
result = mpcNetwork.run(1)
print(result)
print("Day's Total:", sum(result) % 256)

Inputs:
Party 1 input: [10.611561922330747]
Party 2 input: [10.786213091135949]
Party 3 input: [10.758179181169703]
Party 4 input: [10.861105598751285]
Party 5 input: [10.713363225070921]

Start of round 1:
party with id 1 sending to party with id 1 a message with content: 19.62539246901529
party with id 1 sending to party with id 2 a message with content: 167.92368898552246
party with id 1 sending to party with id 3 a message with content: 82.1255112881261
party with id 1 sending to party with id 4 a message with content: 111.62971277987177
party with id 1 sending to party with id 5 a message with content: 141.30725639979516
party with id 2 sending to party with id 1 a message with content: 143.33322932485473
party with id 2 sending to party with id 2 a message with content: 244.93177349844285
party with id 2 sending to party with id 3 a message with content: 60.43520562156094
party with id 2 sending to party with id 4 a message with content: 141.0916851607797
party with id 2 sending 

**Smart Contract**
 
Govern the behaviour of the sensors. (Base Code from HW5)

In [ ]:
pragma solidity ^0.8.0;

contract SensorContract {
    
    address public owner;
    uint256[] private ownDailyTotals;
    uint256 public aggregateDailyTotal;
    uint256 private numberOfSensors;
    uint256 public scheduledTime;
    uint256 public maxCommitment;
    
    mapping(address => bool) private sensors;
    mapping(address => bool) private revealed;

    constructor() {
      owner = msg.sender;
    }

    function pullTotal(uint256 value) private {
      require(sensors[msg.sender], "Sender is not a registered sensor.");
      require(!hasRevealed[msg.sender], "You have already revealed");
      hasRevealed[msg.sender] = true;
      # Pulls from local sensor data (code written in python)
      ownDailyTotal[msg.sender] = SensorCollection(); 
    }


    function runMPC() private {
        require(msg.sender == owner, "Only owner can call MPC input.");
        # Call implemented MPC (code written in python)
        aggregateDailyTotal = runMPC(ownDailyTotal) 
    }


    function sum(uint256[] input) private returns (uint256) {
        uint256 total = 0;
        for (uint256 i = 0; i < input.length; i++) {
            total += input[i];
        }
        return total;
    }

    function postDailyTotal() returns (uint256) {
      require(msg.sender == owner, "Only owner can post a daily total");
      require(block.timestamp >= scheduledTime, "Function cannot be executed yet.");
      require(!hasRevealed[msg.sender], "You have already revealed");
      aggregateDailyTotal = sum(runMPC())
      # Proof generated using external circom code
      proof = generateProof(mpc_inputs, aggregateDailyTotal) 
      hasRevealed[msg.sender] = true;
      return (aggregateDailyTotal, proof)

    }

    function setScheduledTime(uint256 scheduledTime) public {
        require(msg.sender == owner, "Only owner can set max commitment");
        maxCommitment = scheduledTime;
    }

    function setMaxCommitment(uint256 _maxCommitment) public {
        require(msg.sender == owner, "Only owner can set max commitment");
        maxCommitment = _maxCommitment;
    }

    function registerSensor(address sensorAddress) public {
        require(msg.sender == owner, "Only owner can register sensor");
        sensors[sensorAddress] = true;
        numberOfSensors++;
    }

    function unregisterSensor(address sensorAddress) public {
        require(msg.sender == owner, "Only owner can unregister sensor");
        sensors[sensorAddress] = false;
        numberOfSensors--;
    }
}


**ZK Proofs**

Below are two circom circuits written for the purpose of generating ZK proofs and corresponding verifiers with zkREPL. The first circuit is used for the energy consumption and food waste data (the data that is summed for a daily total). The second circuit is used for the temperature data (the data that is averaged daily). Both of these circom circuits can be pasted into the editor at https://zkrepl.dev/ in order to generate the proof/verifier.
 

In [ ]:
# circom code for zk-proof for a+b+c+d+e == f

# In practice these ZK proofs would need to use the PKI to ensure that 
# the sensor data truly came from our sensors. With limited time, we did not 
# implement the PKI. As it stands, this circuit lets us generate a proof that 
# the five input values add up to computed sum without revealing the input 
# values. In practice there would be the additional layer of confidence for 
# the user based on the fact that the proof also proves that the input data 
# came from our sensors.

"""
pragma circom 2.1.4;

include "circomlib/poseidon.circom";

template proveSum () {
    
    signal input sensor1;
    signal input sensor2;
    signal input sensor3;
    signal input sensor4;
    signal input sensor5;
    signal input publicComputedSum;
    signal input randomness;

    signal output h;

    var x[6];
    x[0] = sensor1;
    x[1] = sensor2;
    x[2] = sensor3;
    x[3] = sensor4;
    x[4] = sensor5;
    x[5] = publicComputedSum;

    // Verify authenticity by checking signatures of the sensors, which would 
    // be passed in, using the PKI. This step allows the user to be confident 
    // that the input data used for the proof actually came from the sensors. 
    // The PKI would allow us to get the public key that is associated with the 
    // sensor in order to check the validitity of the signature. This step 
    // convinces the user that the data is indeed real sensor data. 

    signal sum;
    sum <-- (x[0] + x[1] + x[2] + x[3] + x[4]);
    assert(sum == x[5]);

    component com = Poseidon(7);  
    com.inputs[0] <-- sensor1;          
    com.inputs[1] <-- sensor2;
    com.inputs[2] <-- sensor3;  
    com.inputs[3] <-- sensor4;
    com.inputs[4] <-- sensor5;
    com.inputs[5] <-- publicComputedSum;        
    com.inputs[6] <-- randomness;

    h <-- com.out;
}
    component main { public [ publicComputedSum ] } = proveSum();

// recording below a default choice for the inputs

/* INPUT = {
    "sensor1": "1",
    "sensor2": "2",
    "sensor3": "3",
    "sensor4": "4",
    "sensor5": "5",
    "publicComputedSum": "99999999",
    "randomness": "12345"
} */
"""



In [ ]:
#circom code for zk-proof for a+b+c+d+e/5 == f 
# --> This will be used for the temperature data

"""
pragma circom 2.1.4;

include "circomlib/poseidon.circom";

template proveSum () {
    
    signal input sensor1;
    signal input sensor2;
    signal input sensor3;
    signal input sensor4;
    signal input sensor5;
    signal input publicComputedAverage;
    signal input randomness;

    signal output h;

    var x[6];
    x[0] = sensor1;
    x[1] = sensor2;
    x[2] = sensor3;
    x[3] = sensor4;
    x[4] = sensor5;
    x[5] = publicComputedAverage;

    // Verify authenticity by checking signatures of the sensors, which would 
    // be passed in, using the PKI. This step allows the user to be confident 
    // that the input data used for the proof actually came from the sensors. 
    // The PKI would allow us to get the public key that is associated with the 
    // sensor in order to check the validitity of the signature. This step 
    // convinces the user that the data is indeed real sensor data. 

    signal avg;
    avg <-- (x[0] + x[1] + x[2] + x[3] + x[4]) / 5;
    assert(avg == x[5]);

    component com = Poseidon(7);  
    com.inputs[0] <-- sensor1;          
    com.inputs[1] <-- sensor2;
    com.inputs[2] <-- sensor3;  
    com.inputs[3] <-- sensor4;
    com.inputs[4] <-- sensor5;
    com.inputs[5] <-- publicComputedAverage;        
    com.inputs[6] <-- randomness;

    h <-- com.out;
}
    component main { public [ publicComputedAverage ] } = proveSum();

// recording below a default choice for the inputs

/* INPUT = {
    "sensor1": "1",
    "sensor2": "2",
    "sensor3": "3",
    "sensor4": "4",
    "sensor5": "5",
    "publicComputedAverage": "99999999",
    "randomness": "12345"
} */
"""
